In [1]:
import pandas as pd
import numpy as np

## 19년 경기 결과 가져오기

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.request import urlopen


In [3]:
# # selenium 안쓰고 req로 가져오려니까 못가져옴 .. 모르겠..
driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://www.scoreboard.com/kr/soccer/asia/asian-cup/results/') # 새창이 하나 뜰 것


In [4]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# 해당 테이블의 id값들은 soup로 뽑아오기
table_1 = soup.select('#fs-results > table:nth-child(2) > tbody > tr ')
tmp = []
for i in table_1:
    tmp.append(i.attrs['id'])

In [6]:
priod = []
left_team = []
right_team = []
score = []

for i in tmp:

    a = driver.find_element_by_xpath('//*[@id="%s"]/td[2]'%i).text # 각 xpath에 id 부분을 위에서 추출한 id값을 넣어주면 됨
    b = driver.find_element_by_xpath('//*[@id="%s"]/td[3]/span'%i).text
    c = driver.find_element_by_xpath('//*[@id="%s"]/td[4]/span'%i).text
    d = driver.find_element_by_xpath('//*[@id="%s"]/td[5]'%i).text
    
    priod.append(a)
    left_team.append(b)
    right_team.append(c)
    score.append(d)


In [180]:
df= pd.DataFrame({'priod':priod,'left_team':left_team,'right_team':right_team,'score':score})
df.head()
    

,priod,left_team,right_team,score
0,18.01. 01:00,레바논,북한,4 : 1
1,18.01. 01:00,사우디아라비아,카타르,0 : 2
2,17.01. 22:30,오만,투르크메니스탄,3 : 1
3,17.01. 22:30,일본,우즈베키스탄,2 : 1
4,17.01. 01:00,베트남,예멘,2 : 0


In [181]:
# score 나눠주기
left_score = []
right_score = []

for i in df['score']:
    tmp = i.split(':')
    left_score.append(tmp[0])
    right_score.append(tmp[1])
    
df['left_score'] = left_score
df['right_score'] = right_score
df.head()

,priod,left_team,right_team,score,left_score,right_score
0,18.01. 01:00,레바논,북한,4 : 1,4,1
1,18.01. 01:00,사우디아라비아,카타르,0 : 2,0,2
2,17.01. 22:30,오만,투르크메니스탄,3 : 1,3,1
3,17.01. 22:30,일본,우즈베키스탄,2 : 1,2,1
4,17.01. 01:00,베트남,예멘,2 : 0,2,0


In [182]:
# 기존 score 컬럼 삭제
df = df.drop(columns=['score'])
df.head()

,priod,left_team,right_team,left_score,right_score
0,18.01. 01:00,레바논,북한,4,1
1,18.01. 01:00,사우디아라비아,카타르,0,2
2,17.01. 22:30,오만,투르크메니스탄,3,1
3,17.01. 22:30,일본,우즈베키스탄,2,1
4,17.01. 01:00,베트남,예멘,2,0


In [183]:
# int가 아니라 비교가 안됨
df['left_score'] = df['left_score'].astype('int')
df['right_score'] = df['right_score'].astype('int')

In [184]:
for i in range(0,len(df)):
    if df.loc[i,'left_score'] > df.loc[i,'right_score']:
        df.loc[i,'result'] = 2 # 왼쪽팀이 이긴 경우
        
    if df.loc[i,'left_score'] < df.loc[i,'right_score']:
        df.loc[i,'result'] = 0 # 왼쪽팀이 진 경우
    
    if df.loc[i,'left_score'] == df.loc[i,'right_score']:
        df.loc[i,'result'] = 1 # 비긴 경우

In [185]:
df.head()

,priod,left_team,right_team,left_score,right_score,result
0,18.01. 01:00,레바논,북한,4,1,2.0
1,18.01. 01:00,사우디아라비아,카타르,0,2,0.0
2,17.01. 22:30,오만,투르크메니스탄,3,1,2.0
3,17.01. 22:30,일본,우즈베키스탄,2,1,2.0
4,17.01. 01:00,베트남,예멘,2,0,2.0


In [196]:
# 스코어 차이 절대값
for i in range(0,len(df)):
    df.loc[i,'point'] = df.loc[i,'left_score'] - df.loc[i,'right_score']

In [197]:
df.head()

,priod,left_team,right_team,left_score,right_score,result,point
0,18.01. 01:00,레바논,북한,4,1,2,3
1,18.01. 01:00,사우디아라비아,카타르,0,2,0,-2
2,17.01. 22:30,오만,투르크메니스탄,3,1,2,2
3,17.01. 22:30,일본,우즈베키스탄,2,1,2,1
4,17.01. 01:00,베트남,예멘,2,0,2,2


In [188]:
# float에서 int로 변환
df['result'] = df['result'].astype('int')
df['point'] = df['point'].astype('int')

In [189]:
df.head()

,priod,left_team,right_team,left_score,right_score,result,point
0,18.01. 01:00,레바논,북한,4,1,2,3
1,18.01. 01:00,사우디아라비아,카타르,0,2,0,2
2,17.01. 22:30,오만,투르크메니스탄,3,1,2,2
3,17.01. 22:30,일본,우즈베키스탄,2,1,2,1
4,17.01. 01:00,베트남,예멘,2,0,2,2


In [190]:
df2 = df.copy()
df2 = df2.drop(columns=['result','point'])
df2.head()

,priod,left_team,right_team,left_score,right_score
0,18.01. 01:00,레바논,북한,4,1
1,18.01. 01:00,사우디아라비아,카타르,0,2
2,17.01. 22:30,오만,투르크메니스탄,3,1
3,17.01. 22:30,일본,우즈베키스탄,2,1
4,17.01. 01:00,베트남,예멘,2,0


In [191]:
for i in range(0,len(df2)):
    if df2.loc[i,'right_score'] > df2.loc[i,'left_score']:
        df2.loc[i,'result'] = 2 # 오른쪽팀이 이긴 경우
        
    if df2.loc[i,'right_score'] < df2.loc[i,'left_score']:
        df2.loc[i,'result'] = 0 # 오른쪽팀이 진 경우
    
    if df2.loc[i,'right_score'] == df2.loc[i,'left_score']:
        df2.loc[i,'result'] = 1 # 비긴 경우

In [194]:
# 스코어 차이 절대값
for i in range(0,len(df2)):
    df2.loc[i,'point'] = df2.loc[i,'right_score'] - df2.loc[i,'left_score']

In [198]:
# float에서 int로 변환
df2['result'] = df2['result'].astype('int')
df2['point'] = df2['point'].astype('int')

In [199]:
df2.head()

,priod,left_team,right_team,left_score,right_score,result,point
0,18.01. 01:00,레바논,북한,4,1,0,-3
1,18.01. 01:00,사우디아라비아,카타르,0,2,2,2
2,17.01. 22:30,오만,투르크메니스탄,3,1,0,-2
3,17.01. 22:30,일본,우즈베키스탄,2,1,0,-1
4,17.01. 01:00,베트남,예멘,2,0,0,-2


In [201]:
# 컬럼 순서 바꿔주기
df2 = df2.reindex(columns=['priod','right_team','left_team','right_score','left_score','result','point'])
df2.head()


,priod,right_team,left_team,right_score,left_score,result,point
0,18.01. 01:00,북한,레바논,1,4,0,-3
1,18.01. 01:00,카타르,사우디아라비아,2,0,2,2
2,17.01. 22:30,투르크메니스탄,오만,1,3,0,-2
3,17.01. 22:30,우즈베키스탄,일본,1,2,0,-1
4,17.01. 01:00,예멘,베트남,0,2,0,-2


In [203]:
# concat하기 위해 컬럼 이름 바꿔주기
df2 = df2.rename(columns={'right_team':'left_team', 'right_score':'left_score', 'left_team': 'right_team', 'left_score':'right_score'})
df2.head()

,priod,left_team,right_team,left_score,right_score,result,point
0,18.01. 01:00,북한,레바논,1,4,0,-3
1,18.01. 01:00,카타르,사우디아라비아,2,0,2,2
2,17.01. 22:30,투르크메니스탄,오만,1,3,0,-2
3,17.01. 22:30,우즈베키스탄,일본,1,2,0,-1
4,17.01. 01:00,예멘,베트남,0,2,0,-2


In [210]:
df_tot = pd.concat([df,df2])
df_tot= df_tot.drop(columns=['right_team'])
df_tot.head()

,priod,left_team,left_score,right_score,result,point
0,18.01. 01:00,레바논,4,1,2,3
1,18.01. 01:00,사우디아라비아,0,2,0,-2
2,17.01. 22:30,오만,3,1,2,2
3,17.01. 22:30,일본,2,1,2,1
4,17.01. 01:00,베트남,2,0,2,2


In [214]:
df_tot.columns = ['경기날짜', '팀', '팀1점수', '팀2점수','승패','점수차']
df_tot.head()

,경기날짜,팀,팀1점수,팀2점수,승패,점수차
0,18.01. 01:00,레바논,4,1,2,3
1,18.01. 01:00,사우디아라비아,0,2,0,-2
2,17.01. 22:30,오만,3,1,2,2
3,17.01. 22:30,일본,2,1,2,1
4,17.01. 01:00,베트남,2,0,2,2


In [216]:
# 윗부분은 준영씨가
df_tot.to_csv('./data/19year_ver_1_190211.csv', index=False, header=True, encoding='utf-8')

In [219]:
# 준영씨랑 통합본
asiancup19 = pd.read_csv('./data/AsianCup19.csv')
asiancup19.head()

,경기날짜,팀,팀1점수,팀2점수,승패,점수차
0,01.02. 23:00,일본,1,3,0,-2
1,29.01. 23:00,카타르,4,0,2,4
2,28.01. 23:00,이란,0,3,0,-3
3,26.01. 01:00,아랍에미리트,1,0,2,1
4,25.01. 22:00,대한민국,0,1,0,-1


----